In [20]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.dummy import DummyRegressor
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.linear_model import Ridge

In [21]:
!pwd

/Users/manonlaffly/code/cobergmann/MA_PREDICTOR/notebooks/Models


In [22]:
data_orig = pd.read_csv('../../MA_PREDICTOR/data/ma_data_car_clean.csv')

In [23]:
data = data_orig[(data_orig.car > -0.3) & (data_orig.car < 0.3)]

In [24]:
data.drop('month', inplace=True, axis=1)

In [25]:
data

,consideration_offered,shares_acquired,shares_at_announcement,acquiror_code,target_code,target_status,acquisition_count,bidder_count,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target,car
0,Cash,full,no,50102030,50103030,others,15,1,cross_border,business_sector,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,0.006854
1,Other,full,no,54201030,63103010,others,31,1,cross_border,not_related,Consumer Non-Cyclicals,Personal & Household Products & Services,Academic & Educational Services,Academic & Educational Services,-0.010266
2,Other,full,no,57201030,57201020,others,2,1,cross_border,industry_group,Technology,Software & IT Services,Technology,Software & IT Services,0.007746
3,Cash,full,no,52102010,51101010,others,7,1,national,not_related,Industrials,Industrial Goods,Basic Materials,Chemicals,-0.011133
4,Cash,not_full,no,50102030,50102030,public,8,1,cross_border,industry,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,-0.003971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18219,Cash,full,no,52102010,59103010,others,15,1,cross_border,not_related,Industrials,Industrial Goods,Utilities,Utilities,0.020108
18220,Other,full,no,53203020,53205020,others,60,1,cross_border,business_sector,Consumer Cyclicals,Cyclical Consumer Products,Consumer Cyclicals,Cyclical Consumer Products,-0.040156
18221,Other,full,no,54301020,57201010,others,52,1,national,not_related,Consumer Non-Cyclicals,Food & Drug Retailing,Technology,Software & IT Services,-0.003545
18222,Other,full,no,55101010,52203030,others,6,1,cross_border,not_related,Financials,Banking & Investment Services,Industrials,Industrial & Commercial Services,-0.025992


scale the acq count


In [26]:
data.columns

Index(['consideration_offered', 'shares_acquired', 'shares_at_announcement',
       'acquiror_code', 'target_code', 'target_status', 'acquisition_count',
       'bidder_count', 'cross_border', 'relatedness', 'economic_sector_ac',
       'business_sector_ac', 'economic_sector_target',
       'business_sector_target', 'car'],
      dtype='object')

In [27]:
y=data['car']

In [28]:
X=data[['consideration_offered', 'shares_acquired', 'shares_at_announcement',
       'acquiror_code', 'target_code', 'target_status', 'acquisition_count',
       'bidder_count', 'cross_border', 'relatedness', 'economic_sector_ac',
       'business_sector_ac', 'economic_sector_target',
       'business_sector_target']]

# Splitting of the dataset

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

# Pipeline and GridSearch

In [30]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [31]:
num_transformer= MinMaxScaler()

In [32]:
cat_features=['consideration_offered', 'shares_acquired', 'shares_at_announcement', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target', 'target_status', 
       'business_sector_target']

In [33]:
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['acquisition_count']),
    ('cat_transformer', cat_transformer, cat_features)], remainder='passthrough')

## Baseline

In [34]:
baseline_model_mean = DummyRegressor(strategy="mean") 

In [35]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='r2').mean()

-0.0005477136768966328

In [36]:
cross_val_score(baseline_model_mean, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-0.0021237193472723316

## ElasticNet

In [42]:
# Hyperparameter Grid
grid = {'model__alpha': [0.1, 0.2, 0.3, 0.4],
        'model__l1_ratio': [0,0.05, 0.1, 0.15, 0.2],
        'model__max_iter': [10000]}

In [43]:
# Combine preprocessor and linear model in pipeline
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', ElasticNet())])

In [44]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [45]:
# Fit data to Grid Search
search.fit(X_train,y_train);

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13.504938007190573, tolerance: 0.00270848662194968
  model = cd_fast.sparse_enet_coordinate_descent(


In [46]:
# Best score 1
search.best_score_

-0.0021202731621259227

## RandomForest

In [55]:
RandomForestRegressor().get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.823037600169421, tolerance: 0.002171952132502156
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.909977424148362, tolerance: 0.0021893696612592245
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.910393640038457, tolerance: 0.002189369661

In [26]:
# Hyperparameter Grid
grid={'model__bootstrap': [True, False],
      'model__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
      'model__max_features': ['auto', 'sqrt'],
      'model__min_samples_leaf': [1, 2, 4, 10],
      'model__min_samples_split': [2, 5, 10],
      'model__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [27]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', RandomForestRegressor())])

In [28]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [29]:
# Fit data to Grid Search
search.fit(X_train,y_train);

KeyboardInterrupt: 

In [61]:
# Best score 1
search.best_score_

-0.0025065721534876877

In [ ]:
# Best score 2
search.best_score_

In [ ]:
# Best score 3
search.best_score_

In [62]:
# Best Params
search.best_params_

{'model__n_estimators': 150}

## XGBoost

In [51]:
model= XGBRegressor()

In [52]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [53]:
cross_val_score(pipe, X_train, y_train, cv=5, scoring='neg_mean_squared_error').mean()

-0.002359022306046164

In [59]:
# Hyperparameter Grid
grid = {'model__n_estimators': [250, 500, 700],
        'model__learning_rate':  [0.05, 0.10 , 0.15,  0.2],
        'model__max_depth': [1, 2]}

In [73]:
grid = {"model__learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
         "model__max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
         "model__min_child_weight" : [ 1, 3, 5, 7 ],
         "model__gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
         "model__colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

In [74]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [75]:
# Fit data to Grid Search
search.fit(X_train,y_train);

KeyboardInterrupt: 

In [57]:
# Best score 1
search.best_score_

-0.0021217503653858157

In [58]:
search.best_params_

{'model__learning_rate': 0.1,
 'model__max_depth': 1,
 'model__n_estimators': 500}

/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.817080213757949, tolerance: 0.002171952132502156
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.657182290300746, tolerance: 0.002137919167920729
  model = cd_fast.sparse_enet_coordinate_descent(
/Users/manonlaffly/.pyenv/versions/3.8.6/envs/MA_PREDICTOR/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:513: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.916820397435103, tolerance: 0.0021893696612

In [62]:
# Best score 2
search.best_score_

-0.002118488400530452

In [63]:
search.best_params_

{'model__learning_rate': 0.05,
 'model__max_depth': 1,
 'model__n_estimators': 250}

In [ ]:
# Best score 3
search.best_score_

In [ ]:
search.best_params_

## Ridge

In [79]:
model= Ridge()

In [80]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [81]:
# Hyperparameter Grid
grid = {'model__alpha':[200, 230, 250,265, 270, 275, 290, 300, 500]}

In [82]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [83]:
# Fit data to Grid Search
search.fit(X_train,y_train);

In [84]:
# Best score 1
search.best_score_

-0.0021222005287330737

In [85]:
search.best_params_

{'model__alpha': 200}

## SVR

In [34]:
model= SVR()

In [35]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)])

In [36]:
# Hyperparameter Grid
grid = {'model__kernel': ['linear', 'rbf'],
        'model__gamma':  [2e-15, 2e-10, 2e-5, 2, 2e3],
        'model__C': [2e-5,2e-3,2, 2e5, 2e15]}

In [37]:
search = GridSearchCV(pipe, 
                      grid, 
                      scoring = ['neg_mean_squared_error', 'r2','neg_mean_absolute_error'],
                      refit='neg_mean_squared_error',
                      cv = 5,
                      n_jobs=-1) 

In [38]:
# Fit data to Grid Search
search.fit(X_train,y_train);

KeyboardInterrupt: 

In [ ]:
# Best score 1
search.best_score_

In [ ]:
search.best_params_